<a href="https://colab.research.google.com/github/Loopercool/Data-Detectives/blob/main/Master_Data_Detectives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import datetime
import math
from sklearn import preprocessing

import pytz
from datetime import datetime
from sklearn.impute import KNNImputer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/EvCharge/EVChargingStationUsage.csv')
df.head()

<ipython-input-3-61b7df2184e1>:1: DtypeWarning: Columns (29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/EvCharge/EVChargingStationUsage.csv')


,Station Name,MAC Address,Org Name,Start Date,Start Time Zone,End Date,End Time Zone,Transaction Date (Pacific Time),Total Duration (hh:mm:ss),Charging Time (hh:mm:ss),...,Longitude,Currency,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number
0,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/29/2011 20:17,PDT,7/29/2011 23:20,PDT,7/29/2011 23:20,3:03:32,1:54:03,...,-122.160309,USD,0.0,Plug Out at Vehicle,3,95124.0,3284.0,NaN,NaN,NaN
1,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 0:00,PDT,7/30/2011 0:02,PDT,7/30/2011 0:02,0:02:06,0:01:54,...,-122.160309,USD,0.0,Customer,4,94301.0,4169.0,NaN,NaN,NaN
2,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 8:16,PDT,7/30/2011 12:34,PDT,7/30/2011 12:34,4:17:32,4:17:28,...,-122.160309,USD,0.0,Plug Out at Vehicle,5,94301.0,4169.0,NaN,NaN,NaN
3,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 14:51,PDT,7/30/2011 16:55,PDT,7/30/2011 16:55,2:03:24,2:02:58,...,-122.160309,USD,0.0,Customer,6,94302.0,2545.0,NaN,NaN,NaN
4,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 18:51,PDT,7/30/2011 20:03,PDT,7/30/2011 20:03,1:11:24,0:43:54,...,-122.160309,USD,0.0,Plug Out at Vehicle,7,94043.0,3765.0,NaN,NaN,NaN


In [ ]:
print(df.columns)

Index(['Station Name', 'MAC Address', 'Org Name', 'Start Date',
       'Start Time Zone', 'End Date', 'End Time Zone',
       'Transaction Date (Pacific Time)', 'Total Duration (hh:mm:ss)',
       'Charging Time (hh:mm:ss)', 'Energy (kWh)', 'GHG Savings (kg)',
       'Gasoline Savings (gallons)', 'Port Type', 'Port Number', 'Plug Type',
       'EVSE ID', 'Address 1', 'City', 'State/Province', 'Postal Code',
       'Country', 'Latitude', 'Longitude', 'Currency', 'Fee', 'Ended By',
       'Plug In Event Id', 'Driver Postal Code', 'User ID', 'County',
       'System S/N', 'Model Number'],
      dtype='object')


In [ ]:
df.describe()

,Energy (kWh),GHG Savings (kg),Gasoline Savings (gallons),Port Number,EVSE ID,Postal Code,Latitude,Longitude,Fee,Plug In Event Id,Driver Postal Code,System S/N
count,259415.000000,259415.000000,259415.000000,259415.000000,1.804670e+05,259415.000000,259415.000000,259415.000000,259415.000000,2.594150e+05,251013.000000,1.804670e+05
mean,8.544277,3.588596,1.072306,1.591627,1.462973e+05,94302.315768,37.439565,-122.151264,1.097481,1.188685e+08,94170.308255,1.443790e+11
std,7.185462,3.017894,0.901776,0.491534,2.429677e+05,1.941657,0.009621,0.015382,1.934408,4.057750e+08,4249.837126,2.483280e+10
min,0.010000,0.004000,0.001000,1.000000,3.792000e+03,94301.000000,37.422104,-122.163239,0.000000,1.000000e+00,1013.000000,1.023100e+11
25%,3.784000,1.589000,0.475000,1.000000,4.358300e+04,94301.000000,37.428318,-122.162170,0.000000,1.573000e+03,94087.000000,1.231100e+11
50%,6.868634,2.885000,0.862000,2.000000,1.074270e+05,94301.000000,37.444489,-122.158272,0.000000,3.573000e+03,94304.000000,1.427410e+11
75%,11.458340,4.813000,1.438000,2.000000,1.745590e+05,94303.000000,37.446613,-122.145699,1.640000,6.340000e+03,94804.000000,1.705410e+11
max,97.360000,40.891000,12.219000,2.000000,5.535441e+06,94306.000000,37.449375,-122.112480,84.560000,1.609436e+09,99354.000000,2.002410e+11


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259415 entries, 0 to 259414
Data columns (total 33 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Station Name                     259415 non-null  object 
 1   MAC Address                      259415 non-null  object 
 2   Org Name                         259415 non-null  object 
 3   Start Date                       259415 non-null  object 
 4   Start Time Zone                  259415 non-null  object 
 5   End Date                         259415 non-null  object 
 6   End Time Zone                    259415 non-null  object 
 7   Transaction Date (Pacific Time)  259206 non-null  object 
 8   Total Duration (hh:mm:ss)        259415 non-null  object 
 9   Charging Time (hh:mm:ss)         259415 non-null  object 
 10  Energy (kWh)                     259415 non-null  float64
 11  GHG Savings (kg)                 259415 non-null  float64
 12  Ga

In [ ]:
columns_to_drop=['Energy (kWh)','GHG Savings (kg)','Gasoline Savings (gallons)','Currency','Country','Org Name','Plug Type','Port Type','Port Number']
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
print(df.columns)

Index(['Station Name', 'MAC Address', 'Start Date', 'Start Time Zone',
       'End Date', 'End Time Zone', 'Transaction Date (Pacific Time)',
       'Total Duration (hh:mm:ss)', 'Charging Time (hh:mm:ss)', 'EVSE ID',
       'Address 1', 'City', 'State/Province', 'Postal Code', 'Latitude',
       'Longitude', 'Fee', 'Ended By', 'Plug In Event Id',
       'Driver Postal Code', 'User ID', 'County', 'System S/N',
       'Model Number'],
      dtype='object')


In [ ]:
df.shape

(259415, 24)

In [ ]:
column_to_clean = "EVSE ID"  # Replace with the actual column name

# Convert non-numeric values to NaN in the specified column
df[column_to_clean] = pd.to_numeric(df[column_to_clean], errors="coerce")

In [ ]:
feature_to_impute = "EVSE ID"  # Replace with the actual feature name
missing_indices = df[df[feature_to_impute].isnull()].index

# Create a KNNImputer instance
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Extract the feature to impute and perform KNN imputation
feature_data = df[[feature_to_impute]]
imputed_feature = imputer.fit_transform(feature_data)

# Replace the missing values in the original data with imputed values
df.loc[missing_indices, feature_to_impute] = imputed_feature[missing_indices]

In [ ]:
column_to_check = "EVSE ID"  # Replace with the actual column name

# Find and count null values in the specified column
null_values = df[column_to_check].isnull()
count_null = null_values.sum()

print("Null values in column '{}': {}".format(column_to_check, count_null))

Null values in column 'EVSE ID': 0


In [ ]:
# Specify the column to clean
column_to_clean = "Driver Postal Code"  # Replace with the actual column name

# Convert non-numeric values to NaN in the specified column
df[column_to_clean] = pd.to_numeric(df[column_to_clean], errors="coerce")

# Drop rows with NaN values in the specified column
# data.dropna(subset=[column_to_clean], inplace=True)

In [ ]:
feature_to_impute = "Driver Postal Code"  # Replace with the actual feature name
missing_indices = df[df[feature_to_impute].isnull()].index

# Create a KNNImputer instance
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Extract the feature to impute and perform KNN imputation
feature_data = df[[feature_to_impute]]
imputed_feature = imputer.fit_transform(feature_data)

# Replace the missing values in the original data with imputed values
df.loc[missing_indices, feature_to_impute] = imputed_feature[missing_indices]

In [ ]:
# Specify the column to check for null values
column_to_check = "Driver Postal Code"  # Replace with the actual column name

# Find and count null values in the specified column
null_values = df[column_to_check].isnull()
count_null = null_values.sum()

print("Null values in column '{}': {}".format(column_to_check, count_null))

Null values in column 'Driver Postal Code': 0


In [ ]:
feature_to_impute = "System S/N"  # Replace with the actual feature name
missing_indices = df[df[feature_to_impute].isnull()].index

# Create a KNNImputer instance
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Extract the feature to impute and perform KNN imputation
feature_data = df[[feature_to_impute]]
imputed_feature = imputer.fit_transform(feature_data)

# Replace the missing values in the original data with imputed values
df.loc[missing_indices, feature_to_impute] = imputed_feature[missing_indices]

In [ ]:
selected_columns = ["Start Date", "Start Time Zone"]

# Create a new DataFrame with the selected columns
dfa = df[selected_columns]

In [ ]:
# Define time zones
pst = pytz.timezone('US/Pacific')
pdt = pytz.timezone('US/Pacific')

# Function to convert PST to PDT
def convert_pst_to_pdt(date_str):
    dt_object = datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    dt_object = pst.localize(dt_object)
    dt_object_pdt = dt_object.astimezone(pdt)

    return dt_object_pdt

# Function to convert date and time to numeric value
def convert_to_numeric(date_str):
    dt_object = datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    numeric_value = dt_object.timestamp()

    return numeric_value

# Convert PST to PDT and apply numeric conversion
converted_date_times = []
numeric_values = []

for index, row in dfa.iterrows():
    if row["Start Time Zone"] == "PST":
        converted_dt = convert_pst_to_pdt(row["Start Date"])
    else:
        converted_dt = datetime.strptime(row["Start Date"], "%m/%d/%Y %H:%M")

    converted_date_times.append(converted_dt)
    numeric_value = convert_to_numeric(row["Start Date"])
    numeric_values.append(numeric_value)

dfa["ConvertedDateTime"] = converted_date_times
dfa["NumericValue"] = numeric_values

print(dfa)


              Start Date Start Time Zone          ConvertedDateTime  \
0        7/29/2011 20:17             PDT        2011-07-29 20:17:00   
1         7/30/2011 0:00             PDT        2011-07-30 00:00:00   
2         7/30/2011 8:16             PDT        2011-07-30 08:16:00   
3        7/30/2011 14:51             PDT        2011-07-30 14:51:00   
4        7/30/2011 18:51             PDT        2011-07-30 18:51:00   
...                  ...             ...                        ...   
259410  12/31/2020 16:39             PST  2020-12-31 16:39:00-08:00   
259411  12/31/2020 16:48             PST  2020-12-31 16:48:00-08:00   
259412  12/31/2020 17:28             PST  2020-12-31 17:28:00-08:00   
259413  12/31/2020 17:49             PST  2020-12-31 17:49:00-08:00   
259414  12/31/2020 18:19             PST  2020-12-31 18:19:00-08:00   

        NumericValue  
0       1.311971e+09  
1       1.311984e+09  
2       1.312014e+09  
3       1.312037e+09  
4       1.312052e+09  
...      

<ipython-input-11-af567324c9d4>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa["ConvertedDateTime"] = converted_date_times
<ipython-input-11-af567324c9d4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa["NumericValue"] = numeric_values


In [ ]:
dfa.head()

,Start Date,Start Time Zone,ConvertedDateTime,NumericValue
0,7/29/2011 20:17,PDT,2011-07-29 20:17:00,1.311971e+09
1,7/30/2011 0:00,PDT,2011-07-30 00:00:00,1.311984e+09
2,7/30/2011 8:16,PDT,2011-07-30 08:16:00,1.312014e+09
3,7/30/2011 14:51,PDT,2011-07-30 14:51:00,1.312037e+09
4,7/30/2011 18:51,PDT,2011-07-30 18:51:00,1.312052e+09


In [ ]:
df["Start Date"] = dfa["NumericValue"]

In [ ]:
df.head()

,Station Name,MAC Address,Start Date,Start Time Zone,End Date,End Time Zone,Transaction Date (Pacific Time),Total Duration (hh:mm:ss),Charging Time (hh:mm:ss),EVSE ID,...,Latitude,Longitude,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number
0,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,1.311971e+09,PDT,7/29/2011 23:20,PDT,7/29/2011 23:20,3:03:32,1:54:03,NaN,...,37.444572,-122.160309,0.0,Plug Out at Vehicle,3,95124.0,3284.0,NaN,NaN,NaN
1,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,1.311984e+09,PDT,7/30/2011 0:02,PDT,7/30/2011 0:02,0:02:06,0:01:54,NaN,...,37.444572,-122.160309,0.0,Customer,4,94301.0,4169.0,NaN,NaN,NaN
2,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,1.312014e+09,PDT,7/30/2011 12:34,PDT,7/30/2011 12:34,4:17:32,4:17:28,NaN,...,37.444572,-122.160309,0.0,Plug Out at Vehicle,5,94301.0,4169.0,NaN,NaN,NaN
3,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,1.312037e+09,PDT,7/30/2011 16:55,PDT,7/30/2011 16:55,2:03:24,2:02:58,NaN,...,37.444572,-122.160309,0.0,Customer,6,94302.0,2545.0,NaN,NaN,NaN
4,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,1.312052e+09,PDT,7/30/2011 20:03,PDT,7/30/2011 20:03,1:11:24,0:43:54,NaN,...,37.444572,-122.160309,0.0,Plug Out at Vehicle,7,94043.0,3765.0,NaN,NaN,NaN


In [ ]:
print("Original dataset:")
print(df)

# Function to check if a value is a valid date and time format
def is_valid_datetime(value):
    try:
        datetime.strptime(value, '%m/%d/%Y %H:%M')
        return True
    except ValueError:
        return False

# Filter out rows with invalid date and time values
df = df[df['End Date'].apply(is_valid_datetime)]

# Print the dataset after dropping invalid rows
print("\nDataset after dropping invalid rows:")
print(df)

Original dataset:
                          Station Name          MAC Address    Start Date  \
0           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76  1.311971e+09   
1           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76  1.311984e+09   
2           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76  1.312014e+09   
3           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76  1.312037e+09   
4           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76  1.312052e+09   
...                                ...                  ...           ...   
259410  PALO ALTO CA / TED THOMPSON #3  0024:B100:0002:9F81  1.609433e+09   
259411       PALO ALTO CA / WEBSTER #1  0024:B100:0002:4233  1.609433e+09   
259412           PALO ALTO CA / MPL #4  0024:B100:0003:CD37  1.609436e+09   
259413       PALO ALTO CA / WEBSTER #1  0024:B100:0002:4233  1.609437e+09   
259414     PALO ALTO CA / CAMBRIDGE #4  0024:B100:0003:3A0A  1.609439e+09   

       Start Time Zone          End Date End Time Zone  \

In [ ]:
selected_columns = ['End Date', 'End Time Zone']

# Create a new DataFrame with the selected columns
dfb = df[selected_columns]

In [ ]:
import pandas as pd
import pytz
from datetime import datetime

# Define time zones
pst = pytz.timezone('US/Pacific')
pdt = pytz.timezone('US/Pacific')

# Function to convert PST to PDT
def convert_pst_to_pdt(date_str):
    dt_object = datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    dt_object = pst.localize(dt_object)
    dt_object_pdt = dt_object.astimezone(pdt)

    return dt_object_pdt

# Function to convert date and time to numeric value
def convert_to_numeric(date_str):
    dt_object = datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    numeric_value = dt_object.timestamp()

    return numeric_value

# Convert PST to PDT and apply numeric conversion
converted_date_times = []
numeric_values = []

for index, row in dfb.iterrows():
    if row["End Time Zone"] == "PST":
        converted_dt = convert_pst_to_pdt(row["End Date"])
    else:
        converted_dt = datetime.strptime(row["End Date"], "%m/%d/%Y %H:%M")

    converted_date_times.append(converted_dt)
    numeric_value = convert_to_numeric(row["End Date"])
    numeric_values.append(numeric_value)

dfb["ConvertedDateTime"] = converted_date_times
dfb["NumericValue"] = numeric_values

print(dfb)


                End Date End Time Zone          ConvertedDateTime  \
0        7/29/2011 23:20           PDT        2011-07-29 23:20:00   
1         7/30/2011 0:02           PDT        2011-07-30 00:02:00   
2        7/30/2011 12:34           PDT        2011-07-30 12:34:00   
3        7/30/2011 16:55           PDT        2011-07-30 16:55:00   
4        7/30/2011 20:03           PDT        2011-07-30 20:03:00   
...                  ...           ...                        ...   
259410  12/31/2020 17:14           PST  2020-12-31 17:14:00-08:00   
259411  12/31/2020 20:31           PST  2020-12-31 20:31:00-08:00   
259412    1/1/2021 10:56           PST  2021-01-01 10:56:00-08:00   
259413  12/31/2020 21:55           PST  2020-12-31 21:55:00-08:00   
259414  12/31/2020 19:01           PST  2020-12-31 19:01:00-08:00   

        NumericValue  
0       1.311982e+09  
1       1.311984e+09  
2       1.312029e+09  
3       1.312045e+09  
4       1.312056e+09  
...              ...  
259410  1.

<ipython-input-17-33177a2b38be>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb["ConvertedDateTime"] = converted_date_times
<ipython-input-17-33177a2b38be>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb["NumericValue"] = numeric_values


In [ ]:
df["End Date"] = dfa["NumericValue"]

In [ ]:
columns_to_drop=['Start Time Zone','End Time Zone','Transaction Date (Pacific Time)']
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df.columns

Index(['Station Name', 'MAC Address', 'Start Date', 'End Date',
       'Total Duration (hh:mm:ss)', 'Charging Time (hh:mm:ss)', 'EVSE ID',
       'Address 1', 'City', 'State/Province', 'Postal Code', 'Latitude',
       'Longitude', 'Fee', 'Ended By', 'Plug In Event Id',
       'Driver Postal Code', 'User ID', 'County', 'System S/N',
       'Model Number'],
      dtype='object')

In [ ]:
def time_to_numeric(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    total_hours = hours + minutes / 60 + seconds / 3600
    return total_hours

df['Total_time_hours'] = df['Total Duration (hh:mm:ss)'].apply(time_to_numeric)

df['Charging_time_hours'] = df['Charging Time (hh:mm:ss)'].apply(time_to_numeric)

In [ ]:
columns_to_drop=['Total Duration (hh:mm:ss)','Charging Time (hh:mm:ss)']
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df.columns

Index(['Station Name', 'MAC Address', 'Start Date', 'End Date', 'EVSE ID',
       'Address 1', 'City', 'State/Province', 'Postal Code', 'Latitude',
       'Longitude', 'Fee', 'Ended By', 'Plug In Event Id',
       'Driver Postal Code', 'User ID', 'County', 'System S/N', 'Model Number',
       'Total_time_hours', 'Charging_time_hours'],
      dtype='object')

# **Replace using KNNs**

## **Replacing User ID**

## **Replacing Driver Postal Code**

## **Replacing System S/N**

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer

feature_to_impute = "System S/N"  # Replace with the actual feature name
missing_indices = df_main[df_main[feature_to_impute].isnull()].index

# Create a KNNImputer instance
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Extract the feature to impute and perform KNN imputation
feature_data = df_main[[feature_to_impute]]
imputed_feature = imputer.fit_transform(feature_data)

# Replace the missing values in the original data with imputed values
df_main.loc[missing_indices, feature_to_impute] = imputed_feature[missing_indices]

In [ ]:
import pandas as pd

# Specify the column to check for null values
column_to_check = "System S/N"  # Replace with the actual column name

# Find and count null values in the specified column
null_values = df_main[column_to_check].isnull()
count_null = null_values.sum()

print("Null values in column '{}': {}".format(column_to_check, count_null))


Null values in column 'System S/N': 0


## **Replacing Model Number**

In [ ]:
import pandas as pd

# Specify the column to clean
column_to_clean = "Model Number"  # Replace with the actual column name

# Convert non-numeric values to NaN in the specified column
df_main[column_to_clean] = pd.to_numeric(df_main[column_to_clean], errors="coerce")

# Drop rows with NaN values in the specified column
# data.dropna(subset=[column_to_clean], inplace=True)

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer

feature_to_impute = "Model Number"  # Replace with the actual feature name
missing_indices = df_main[df_main[feature_to_impute].isnull()].index

# Create a KNNImputer instance
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Extract the feature to impute and perform KNN imputation
feature_data = df_main[[feature_to_impute]]
imputed_feature = imputer.fit_transform(feature_data)

# Replace the missing values in the original data with imputed values
df_main.loc[missing_indices, feature_to_impute] = imputed_feature[missing_indices]

ValueError: ignored

In [ ]:
import pandas as pd

# Specify the column to check for null values
column_to_check = "Model Number"  # Replace with the actual column name

# Find and count null values in the specified column
null_values = df_main[column_to_check].isnull()
count_null = null_values.sum()

print("Null values in column '{}': {}".format(column_to_check, count_null))
